In [46]:
import datasets
from datasets import load_dataset, Dataset
from pathlib import Path
from transformers import GPT2Tokenizer, GPT2Model,  AutoModelForQuestionAnswering, AutoTokenizer
from transformers import Trainer, TrainingArguments

project_dir = Path().cwd().parent
data_dir = project_dir.joinpath("data")

In [56]:
# fist start with trivia
# datasets.config.DOWNLOADED_DATASETS_PATH = "/home/public/wjang/data/triviaqa-wiki"
# datasets.config.DOWNLOADED_DATASETS_PATH = data_dir

train = load_dataset('trivia_qa', 'unfiltered', split="train[:8000]")
test = load_dataset('trivia_qa', 'unfiltered', split='train[8000:10000]')
# test = load_dataset('trivia_qa', 'unfiltered', split ='test[:200]')


Found cached dataset trivia_qa (/home/wonseok/.cache/huggingface/datasets/trivia_qa/unfiltered/1.2.0/ee76d8a9403e71177e2a3fa7e414d1ee28a79a0970d9176f62f268798aa64b31)
Found cached dataset trivia_qa (/home/wonseok/.cache/huggingface/datasets/trivia_qa/unfiltered/1.2.0/ee76d8a9403e71177e2a3fa7e414d1ee28a79a0970d9176f62f268798aa64b31)


In [7]:
# save files

train.to_json(data_dir.joinpath('raw/triviaqa_raw_train.json'))
test.to_json(data_dir.joinpath('raw/triviaqa_raw_test.json'))

NameError: name 'train' is not defined

In [8]:
train = load_dataset('json', data_files=data_dir.joinpath('raw/triviaqa_raw_train.json').as_posix(), split='train')
test = load_dataset('json', data_files=data_dir.joinpath('raw/triviaqa_raw_test.json').as_posix(), split='train')

Found cached dataset json (/home/wonseok/.cache/huggingface/datasets/json/default-62b5c3c6749fb4c0/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Found cached dataset json (/home/wonseok/.cache/huggingface/datasets/json/default-e502800c41ae9e81/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [12]:
context = [source['wiki_context'][0] if len(source['wiki_context']) > 0 else 'no context'for source in train['entity_pages'] ]

In [13]:
from tokenizers.processors import TemplateProcessing

model = AutoModelForQuestionAnswering.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained("gpt2")

bos = '<|bos|>'
eos = '<|eos|>'
pad = '<|pad|>'
sep = '<|sep|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.post_processor = TemplateProcessing(
    single=bos + " $A " + eos,
    special_tokens=[(eos, tokenizer.eos_token_id), (bos, tokenizer.bos_token_id)],
)

# need to resize model vocabulary
model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50261, 768)

In [18]:
inputs = tokenizer(context[0:2], return_offsets_mapping=True, max_length=1024)

In [32]:
tokenizer(context[0:2], max_length=1024, truncation=True, return_token_type_ids=True, )

{'input_ids': [[6435, 37555, 318, 257, 11150, 3474, 4445, 290, 3502, 1605, 9048, 10283, 3194, 290, 18542, 416, 7516, 337, 13, 3059, 37314, 11, 543, 4966, 422, 3267, 362, 11, 11445, 11, 284, 3945, 1511, 11, 4751, 11, 8282, 287, 302, 48381, 20875, 13, 383, 10283, 318, 262, 749, 2968, 290, 14212, 287, 262, 2106, 286, 9048, 22670, 11, 351, 1596, 11, 4531, 22, 22670, 3199, 287, 477, 11, 1642, 340, 366, 853, 14632, 262, 14069, 1621, 1683, 1297, 416, 530, 1692, 852, 1911, 220, 1629, 663, 9103, 11, 2631, 37555, 4966, 287, 625, 362, 11, 8054, 14741, 11, 351, 257, 7183, 1056, 286, 36561, 1510, 287, 5441, 2678, 11, 290, 373, 14251, 656, 2310, 8950, 13, 220, 632, 4193, 284, 20534, 262, 1440, 12, 35330, 32261, 10283, 355, 262, 3210, 287, 262, 1578, 1829, 11, 220, 290, 1978, 351, 663, 20813, 7366, 3059, 37314, 517, 621, 720, 16, 2997, 13, 1432, 22272, 82, 286, 262, 10283, 389, 991, 11150, 3474, 290, 1057, 287, 2048, 790, 471, 13, 50, 13, 7533, 13, 198, 198, 464, 10283, 13692, 5000, 319, 257, 28685, 

In [39]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    wiki_contexts = [source['wiki_context'][0].strip() if len(source['wiki_context']) > 0 else 'no context'.strip() for source in examples['entity_pages'] ]

    inputs = tokenizer(
        questions,
        wiki_contexts,
        max_length=1024,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    # offset_mapping = inputs.pop("offset_mapping")
    # answers = examples["answers"]
    # start_positions = []
    # end_positions = []

    # for i, offset in enumerate(offset_mapping):
    #     # answer = answers[i]
    #     # start_char = answer["answer_start"][0]
    #     # end_char = answer["answer_start"][0] + len(answer["text"][0])
    #     sequence_ids = inputs.sequence_ids(i)

    #     # Find the start and end of the context
    #     idx = 0
    #     while sequence_ids[idx] != 1:
    #         idx += 1
    #     context_start = idx

    #     while sequence_ids[idx] == 1:
    #         idx += 1
    #     context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
    #     if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
    #         start_positions.append(0)
    #         end_positions.append(0)
    #     else:
    #         # Otherwise it's the start and end token positions
    #         idx = context_start
    #         while idx <= context_end and offset[idx][0] <= start_char:
    #             idx += 1
    #         start_positions.append(idx - 1)

    #         idx = context_end
    #         while idx >= context_start and offset[idx][1] >= end_char:
    #             idx -= 1
    #         end_positions.append(idx + 1)

    # inputs["start_positions"] = start_positions
    # inputs["end_positions"] = end_positions
    return inputs

In [45]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/wonseok/.cache/huggingface/datasets/json/default-62b5c3c6749fb4c0/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-e8786452bd4631e7.arrow


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [48]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [50]:
training_args = TrainingArguments(
    output_dir="../model/trivia_gpt",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]

In [74]:
context = dataset['train'][0]['entity_pages']['wiki_context'][0]

In [135]:
inputs = tokenizer(question, context, return_tensors='pt', return_token_type_ids=True, max_length=1024, truncation=True)

In [136]:
import torch
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-3.1998, -9.9643, -8.0330,  ..., -6.3425, -5.7444, -6.0709]]), end_logits=tensor([[-0.0966, -0.7796, -0.9979,  ..., -1.6981, -1.8853, -1.9204]]), hidden_states=None, attentions=None)


In [48]:
outputs.start_logits

tensor([[ 2.3874,  6.4358,  5.6925,  5.6773,  5.6681,  5.2134,  5.1008,  5.8855,
          5.3893,  5.8450,  5.3206,  5.4511,  5.6941,  5.4772,  5.1780,  5.5038,
          5.4234,  4.6833,  4.7428,  4.3512,  5.7624,  4.6304,  3.8184,  3.7696,
          4.0553,  3.6392,  4.7540,  4.3875,  3.5800,  3.8620,  3.5720,  3.4407,
          3.1645,  3.3903,  4.7443,  5.5327,  4.2520,  3.7863,  2.0929,  1.4395,
          4.9716,  3.3676,  3.4566,  3.7941,  4.2121,  2.7184,  1.9635,  1.9507,
          4.4011,  2.8790,  2.5880,  2.7406,  0.9453,  3.2911,  2.6771,  3.8936,
          5.1057,  2.8449,  4.4624,  3.8717,  4.5720,  4.3178,  5.1858,  4.0228,
          3.8016,  4.3358,  4.6010,  4.0006,  3.4059,  4.0620,  3.5027,  4.5354,
          4.6953,  5.2006,  5.0762,  3.5061,  4.3643,  4.9156,  4.7615,  4.4697,
          4.1841,  3.9989,  4.2213,  3.8873,  4.2942,  2.3721,  4.0536,  3.2292,
          3.0520,  3.4255,  2.8044,  3.4301,  4.2049,  3.5849,  3.4642,  4.4165,
          5.4953,  5.9342,  